<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 3b. Cross-Modal Projection

Earlier in this course, we covered a variety of different matrix algebra operations to develop multimodal model architecture. In this lab, we will be exploring a relatively advanced but powerful technique: cross-modal projection. While this technique is commonly used to project image embeddings into a text embedding space, it is not limited to this data type. This time, we will try an opposite approach. Can we project text embeddings into the image embedding space?

#### Learning Objectives

The goals of this notebook are to:
* Investigate the cross-model projector architecture
* Train a cross-model projector
* Integrate a cross-model projector into an existing model

Let's load some libraries to get started.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image
import pandas as pd
import clip

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## 3.1 Setup
### 3.1.1 The Dataset

To project one mode onto another, it would be useful to have a model as base. Let's build a convolutional neural network to classify images while we explore our dataset. We can use [VGG16](https://pytorch.org/vision/main/models/generated/torchvision.models.vgg16.html), a pre-trained image classifier, as a base.

In [ ]:
# load the VGG16 network *pre-trained* on the ImageNet dataset
weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)
vgg_model.requires_grad_(False) # Freeze model during training

VGG16 comes with a set of [torchvision transforms](https://pytorch.org/vision/0.9/transforms.html) we can use to convert images into the shape and format VGG16 can ingest.

In [ ]:
pre_trans = weights.transforms()
pre_trans

Before applying these transformations to all of our data, let's test it on one example. We have a few different flower photos located in the `data/flower_photos/` folder. These photos were taken at the [Elizabeth F. Gamble Garden](https://www.gamblegarden.org/visit-us/overview/) in Palo Alto. Please visit if you get the chance!

<center><img src="data/flower_photos/PXL_20250104_202132415.jpg" width="25%"/></center>

In [ ]:
img_path = "data/flower_photos/PXL_20250104_202132415.jpg"
image = Image.open(img_path)

The following code will apply the transforms and print out the result. Because the image is being normalized, the result will look a bit abstract.

In [ ]:
print('Original image shape: ', mpimg.imread(img_path).shape)
processed_image = tv_io.read_image(img_path).to(device)
processed_image = pre_trans(image)  # weights.transforms()
print("Processed image shape: ", processed_image.shape)
pil_image = torchvision.transforms.functional.to_pil_image(processed_image)
pil_image

Speaking of image transforms, let's define a little bit of data augmentation. We'll randomly [crop](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomResizedCrop.html) and [flip](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomHorizontalFlip.html) the image to add more variety to our dataset.

In [ ]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

random_trans = transforms.Compose([
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(.8, 1), ratio=(1, 1)),
    transforms.RandomHorizontalFlip()
])

We can scale these steps for all of our flower photos. Let's look at the data:

In [ ]:
DATA_DIR = 'data/'
df = pd.read_csv(DATA_DIR + 'flower_photos.csv')
df

There are `3` different kinds of flowers in our dataset:
* [Kniphofia Uvaria](https://en.wikipedia.org/wiki/Kniphofia_uvaria)
* [Salvia Splendens](https://en.wikipedia.org/wiki/Salvia_splendens)
* [Tagetes Patula](https://en.wikipedia.org/wiki/Tagetes_patula)

This dataset is not very large, and that's ok. Because we're using a pre-trained model, the model will have enough prior context in order to learn from these images. We will also purposely overfit this data to verify to architecture can learn on this kind of data. Once we see that the model is learning, we can expand our dataset and create validation data.

In [ ]:
patch_length = 150
patches = 3

class MyDataset(Dataset):
    def __init__(self):
        self.imgs = []
        self.descriptions = []
        self.labels = []
        
        for idx in range(len(df)):
            row = df.loc[idx]
            img = Image.open(DATA_DIR + "flower_photos/" + row["Filename"])
            self.imgs.append(pre_trans(img).to(device))
            text_patches = [row["Description"][t*patch_length:(t+1)*patch_length] for t in range(patches)]
            self.descriptions.append(text_patches)
            label = torch.tensor(row["Class ID"]).to(device)
            self.labels.append(label)


    def __getitem__(self, idx):
        img = self.imgs[idx]
        description = self.descriptions[idx]
        label = self.labels[idx]
        return img, description, label

    def __len__(self):
        return len(self.imgs)

Now that we've defined our dataset, let's initialize it.

In [ ]:
BATCH_SIZE = 32
train_data = MyDataset()
dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
train_N = len(dataloader.dataset)

### 3.1.2 The Image Classification Model

Time to make our image classification model. We'll use most of the VGG16 model, but we'll remove the last few layers and add our own instead. The vector output of the VGG16 portion of the model will become our `embedder` before being fed to the rest of the model.`

To make it easier for other developers to use this embedding, we will define a `get_img_embs` function.

In [ ]:
N_CLASSES = 3

class FlowerClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedder = nn.Sequential(
            vgg_model.features,
            vgg_model.avgpool,
            nn.Flatten(),
            vgg_model.classifier[0:3]
        )
        self.classifier = nn.Sequential(
            nn.Linear(4096, 500),
            nn.ReLU(),
            nn.Linear(500, N_CLASSES)
        )

    def get_img_embs(self, imgs):
        return self.embedder(imgs)
    
    def forward(self, imgs=None, img_embs=None):
        assert (imgs is not None or img_embs is not None), "No images or embeddings given."
        if imgs is not None:
            img_embs = self.get_img_embs(imgs)
        return self.classifier(img_embs)

In [ ]:
flower_classifier = FlowerClassifier().to(device)

Almost done. Let's define out loss function. Since we're classifying these photos into multiple categories, we should use the [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) function.

In [ ]:
def get_classifier_loss(model, batch):
    imgs, _, labels = batch
    pred_labels = model(imgs=random_trans(imgs))
    accuracy = get_batch_accuracy(pred_labels, labels, train_N)
    return nn.CrossEntropyLoss()(pred_labels, labels), accuracy

In `get_classifier_loss`, there is a yet-to-be-defined `get_batch_accuracy` function. Let's define that now. It'll be easier to tell how well the model is performing with the accuracy function instead of the loss function.

In [ ]:
def get_batch_accuracy(pred, label, N):
    y = pred.argmax(dim=-1)
    correct = y.eq(label).sum().item()
    return correct / N

Finally, let's define the function to train the model. We'll be reusing this function to train multiple models, so let's make it flexible by adding the loss function (`loss_fn`) as a parameter.

In [ ]:
def train_model(model, loss_fn, epochs=20, print_accuracy=False):
    lrate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=lrate)

    model.train()
    for epoch in range(epochs):
        accuracy = 0
        batch_accuracy = 0
        for step, batch in enumerate(dataloader):
            optimizer.zero_grad()
            loss, batch_accuracy = loss_fn(model, batch)
            accuracy += batch_accuracy
            loss.backward()
            optimizer.step()

        out_string = f"Epoch {epoch:3d} | Loss: {loss.item():2.4f}"
        if print_accuracy:
            out_string += f" | Accuracy {accuracy:2.4f}"
        print(out_string)
    
    model.eval()

Moment of truth! Let's see how well this classifier learns.

In [ ]:
train_model(flower_classifier, get_classifier_loss, epochs=10, print_accuracy=True)

Due to the randomness behind model training, different runs can have different results. When we ran it, we got an accuracy of about `0.9697`, which is about `96.97%`. Not bad! This will be our benchmark going forward. Let's see if we can change the modality of this model while maintaining this accuracy.

## 3.2 Cross-Modal Projection

Now that we have a base model, let's see how we can change its modality. If a model that projects image embeddings onto text embeddings is called a Vision Language Model (VLM), let's call a model that projects text embeddings onto image embeddings a Language Vision Model (LVM).

To start, let's find a way to get text embeddings. Since it is already designed to correlate text and images, let's use [CLIP](https://github.com/openai/CLIP).

In [ ]:
clip_model, clip_preprocess = clip.load("ViT-B/32")
clip_model.eval()
CLIP_FEATURES = 512

Before we create a model with CLIP, let's get some practice. First, we'll create a list of strings:

In [ ]:
text_list = [
    "A Kniphofia Uvaria flower",
    "A Salvia Splendens flower",
    "A Tagetes Patula flower"
]

Then, we'll run it through CLIP to get an encoding. In this case, we'll use our encoding as our embedding.

In [ ]:
text_tokens = clip.tokenize(text_list).to(device)
clip_text_encodings = clip_model.encode_text(text_tokens).float()
clip_text_encodings

To project CLIP embeddings onto VGG16 embeddings, we should note both the CLIP embedding size and the VGG16 embedding size.

In [ ]:
clip_emb_size = clip_text_encodings[0].shape
clip_emb_size

CLIP has a maximum token length of 77, but our text descriptions of our flowers are much larger than that. In order to add more information, we'll break our text descriptions down into `patches`. Not only does it allow us to analyze a larger description, but we can also spatially break down the reasoning of our text.

Because of this patching, we will run CLIP multiple times per row of description in our dataset. Let's make a function (`get_clip_encodings`) to run through the list of patches. We can use python [list comprehension](https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions) to make the operation independent of the patch size. Then, we can take our list of encodings and concatenate them together.

In [ ]:
def get_clip_encodings(texts):
    text_tokens = [clip.tokenize(t).to(device) for t in texts]
    clip_text_encodings = [clip_model.encode_text(t).float() for t in text_tokens]
    return torch.cat(clip_text_encodings, dim=1)

There are multiple ways we could deduce the VGG16 embedding size. We could look at the model summary. Another way is to feed an image through our `get_img_embs` and check the size of the output. We can batch our `processed_image` with [unsqueeze](https://pytorch.org/docs/main/generated/torch.unsqueeze.html) to test it out.

In [ ]:
test_image = torch.unsqueeze(processed_image, 0).to(device)
imb_embs = flower_classifier.get_img_embs(test_image)
vgg_shape = imb_embs[0].shape
vgg_shape

Now that we know the size we're converting from and the size we're converting to, we can make our modal `projector`. We can use any neural network operations we want as long as our input dimensions and our output dimensions reflect this conversion. It may be surprising to see we only need a few linear layers. Even state-of-the-art Vision Language Models like LLaVA only use a few linear layers in [their projector](https://github.com/huggingface/transformers/blob/v4.49.0/src/transformers/models/llava/modeling_llava.py#L88).

In [ ]:
projector = nn.Sequential(
    nn.Linear(clip_emb_size[0]*patches, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, vgg_shape[0])
).to(device)

We will use our image-description pairs to train the projector. The text description will be fed through CLIP and the projector while the corresponding image will be fed through the `flower_classifier`'s `get_img_embs` function.

Since we're trying to minimize the distance between our predicted image embeddings and the true image embeddings, our loss function will change. Even though we're working in an abstract image and text space, the math is not much different than our robotics position predictor from earlier in the course. Like in those previous labs, we will use the [Mean Squared Error](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html) loss function.

In [ ]:
def get_projector_loss(model, batch):
    imgs, texts, _ = batch
    imb_embs = flower_classifier.get_img_embs(imgs)

    text_encodings = get_clip_encodings(texts)
    pred_img_embs = model(text_encodings).to(device)
    return nn.MSELoss()(pred_img_embs, imb_embs), 0

Time to train the model! Let's see how well we can project these text embeddings into image embeddings.

In [ ]:
train_model(projector, get_projector_loss, epochs=50)

## 3.3 A Language Vision Model

It looks like a projector model was able to learn something, but it is difficult to say how accurate its predictions are. Let's connect it to our `flower_classifier` in order to test how much it learned.`

We can create a new model `MyLVM` to feed the output of our projector as image embeddings in our classifier.

In [ ]:
class MyLVM(nn.Module):
    def __init__(self):
        super().__init__()
        self.projector = projector
        self.flower_classifier = flower_classifier
    
    def forward(self, texts):
        text_encodings = get_clip_encodings(texts)
        proj_img_embs = self.projector(text_encodings).to(device)
        return self.flower_classifier(img_embs=proj_img_embs)

In [ ]:
my_lvm = MyLVM()

Now that we're back to using a classifier model, let's update our loss function. This time, we'll take our text descriptions and feed it into our model to see how well it can identify the correct flower.

In [ ]:
def get_LVM_loss(model, batch):
    _, texts, labels = batch
    pred_labels = model(texts)
    accuracy = get_batch_accuracy(pred_labels, labels, train_N)
    return nn.CrossEntropyLoss()(pred_labels, labels), accuracy

Before we do anymore training, let's see how the model does as is. We'll one through one epoch of the dataset to calculate our total accuracy.

In [ ]:
my_lvm.eval()
accuracy = 0
batch_accuracy = 0
for step, batch in enumerate(dataloader):
    loss, batch_accuracy = get_LVM_loss(my_lvm, batch)
    accuracy += batch_accuracy

out_string = f"Loss: {loss.item():2.4f} | Accuracy {accuracy:2.4f}"
print(out_string) 

Not bad again! Even though the loss of the projector isn't perfect, it does a good enough job for the `flower_classifier` to understand what is being passed to it.

In the paper [Vision Instruction Tuning](https://arxiv.org/abs/2304.08485), it describes a two step approach to training a cross-modal projection model. The first-step we already did: training a projector on embedding pairs. The second step is to fine-tune the entire VLM. Let's replicate the second step for our LVM. This trains the whole pipeline from the inputed text to the output flower prediction.

In [ ]:
train_model(my_lvm, get_LVM_loss, epochs=30, print_accuracy=True)

## Next

Congrats on finishing the lab! This concludes the more mathematical theory section of the course. In the next few labs, we will be reviewing how multi-modal models are used in practice. We're going to need a lot of resources to run these models. Please restart the kernel before continuing.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>